# 2359. Find Closest Node to Given Two nodes



<img src="/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/스크린샷 2023-09-18 오후 2.38.11.png">

In [61]:
from typing import List, Tuple, Set
###################### directed cycle 찾는 함수 #######################  
def detect_cycles(graph: dict) -> List[List[int]]:
    """Returns a list of cycles detected in the graph."""
    visited = set()
    stack = []
    cycles = []

    def dfs(node, parent):
        if node in stack:
            cycle_start_idx = stack.index(node)
            cycles.append(stack[cycle_start_idx:])
            return
        if node in visited:
            return
        visited.add(node)
        stack.append(node)
        for neighbor, _ in graph.get(node, []):
            if neighbor != parent:
                dfs(neighbor, node)
        stack.pop()

    for node in graph:
        if node not in visited:
            dfs(node, None)

    return cycles


def bellman_ford(graph, s):
    distances = {}
    predecessor = {}
    
    # 초기 거리를 모두 무한대로 설정
    for key in graph.keys():
        distances[key] = float('inf')
    distances[s] = 0  # 시작점의 거리는 0으로 설정
    
    # Relaxation 작업 (모든 노드에 대해 (n-1)번 반복하여 거리를 업데이트)
    for _ in range(len(graph) - 1):
        for u in graph:
            for v, w in graph[u]:
                # 더 짧은 경로 발견 시 업데이트
                if distances[u] + w < distances[v]:
                    distances[v] = distances[u] + w
                    predecessor[v] = u

    # Negative cycle 확인 (더 짧은 경로가 발견되면 음수 가중치 순환 있음)
    for u in graph:
        for v, w in graph[u]:
            if distances[u] + w < distances[v]:
                # raise Exception("Negative weight cycle detected")
                return {}, {}
    
    return distances, predecessor

##### positive cycle은 shortest path에 영향을 주지 않으므로 굳이 제외 안 해도 되지만 혹시 몰라 넣어놓음 #####
    # # Negative cycle 확인 + cycle있는 노드 제외
    # for u in graph:
    #     for v, w in graph[u]:
    #         if v in ignore_nodes:  # 순환에 포함된 노드는 무시
    #             continue
    #         if distances[u] + w < distances[v]:
    #             raise Exception("Negative weight cycle detected")

    # return distances, predecessor


def closestMeetingNodes(edges: List[int], node1: int, node2: int) -> int:
    # edge, weight 정보 담은 graph dict
    G = dict()
    # 0번 노드부터니까
    for i in range(len(edges)):
        G[i] = []
    
    # 정보 입력 
    for i in range(len(edges)):
        from_ = i
        if edges[i] != -1:
            to = edges[i]
            weight = 1
        
            G[from_].append((to, weight))
        
    print("Graph: ", G)

######## directed cycle 찾고 해당 nodes를 set에 넣어놓는 함수 ##########  
    # 순환 경로를 탐지
    cycles = detect_cycles(G)

    # 순환에 포함된 모든 노드들의 세트를 구성
    ignore_nodes = set()
    for cycle in cycles:
        for node in cycle:
            ignore_nodes.add(node)
#################################################################
    
    max_dist = dict()
    
    for source in G.keys():
        if source == node1 or source == node2:
            distances, predecoessor = bellman_ford(G, source)
            
            print("minimum distances: ", distances)
            
            for key, val in distances.items():
                if key not in max_dist:
                    max_dist[key] = [val]
                else:
                    max_dist[key].append(val)
                    
    # 두 노드에서 출발하는 것 중 max distance만 저장하기
    for s, vals in max_dist.items():
        if float('inf') in vals:
            max_dist[s] = []
        else:
            dist = max(vals)
            max_dist[s] = dist
    
    print (max_dist)
    
    # max distances 중 가장 smallest 찾기
    smallest_idxNodes = []
    smallest = 100
    for dist in max_dist.values():
        if dist != [] and smallest > dist:
            smallest = dist
    
    # max dist 중 smallest 값인 모든 nodes list에 다 넣고 
    # 맨 처음 node 반환하기
    for k, d in max_dist.items():
        if d == smallest:
            smallest_idxNodes.append(k)
    
    return smallest_idxNodes[0]
        
            

    

In [62]:
edges = [2,2,3,-1]
node1 = 0
node2 = 1
closestMeetingNodes(edges, node1, node2)

Graph:  {0: [(2, 1)], 1: [(2, 1)], 2: [(3, 1)], 3: []}
minimum distances:  {0: 0, 1: inf, 2: 1, 3: 2}
minimum distances:  {0: inf, 1: 0, 2: 1, 3: 2}
{0: [], 1: [], 2: 1, 3: 2}


2

In [60]:
edges = [1,2,-1]
node1 = 0
node2 = 2
closestMeetingNodes(edges, node1, node2)

Graph:  {0: [(1, 1)], 1: [(2, 1)], 2: []}
minimum distances:  {0: 0, 1: 1, 2: 2}
minimum distances:  {0: inf, 1: inf, 2: 0}
{0: [], 1: [], 2: 2}


2

# 743. Network Delay Time



<img src="/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/스크린샷 2023-09-14 오후 8.17.30.png">

In [ ]:
"""
[전체노드 visit되는 최소한의 시간 구하기 (Directed graph with weight)]
-> BFS로 풀기 

You are given a network of n nodes, labeled from 1 to n. 
You are also given times, a list of travel times as directed edges times[i] = (ui, vi, wi), 
where ui is the source node, vi is the target node, 
and wi is the time it takes for a signal to travel from source to target.

We will send a signal from a given node k. 
Return the minimum time it takes for all the n nodes to receive the signal. 
If it is impossible for all the n nodes to receive the signal, return -1.
"""

In [149]:
from typing import List

# DFS + Relaxation
def shortestpath(G, s):
    times = dict()
    
    for key in G.keys():
        if key not in times:
            times[key] = float('inf')
            
    times[s] = 0
    stack = [s]
    
    while stack:
        cur = stack.pop()
        for node, weight in G[cur]:
            if times[node] > times[cur] + weight:
                times[node] = times[cur] + weight
                stack.append(node)
    return times
        
def networkDelayTime(times: List[List[int]], n: int, k: int):
    # edge, weight 정보 담은 graph dict
    G = dict()
    # 1번 노드부터니까
    for i in range(1, n+1):
        G[i] = []
    
    # 정보 입력 
    for i in range(len(times)):
        from_ = times[i][0]
        to = times[i][1]
        weight = times[i][2]
        
        G[from_].append((to, weight))
    print("Graph: ", G)
    
    paths = shortestpath(G, k)
    print("Shortest path: ", paths)
    
    # 모든 노드가 시그널을 받는 데 필요한 시간을 찾아야 합니다.
    max_time = -1
    for time in paths.values():
        if time == float('inf'):
            return -1
        max_time = max(max_time, time)
    
    return max_time


In [150]:
times = [[1,2,1],[2,1,3]]
n = 2
k = 2
networkDelayTime(times, n, k)

Graph:  {1: [(2, 1)], 2: [(1, 3)]}
Shortest path:  {1: 3, 2: 0}


3

In [151]:
times = [[1,2,1],[2,3,2],[1,3,2]]
n = 3
k = 1
networkDelayTime(times, n, k)

Graph:  {1: [(2, 1), (3, 2)], 2: [(3, 2)], 3: []}
Shortest path:  {1: 0, 2: 1, 3: 2}


2

In [152]:
times = [[2,1,1],[2,3,1],[3,4,1]]
n = 4
k = 2
networkDelayTime(times, n, k)

Graph:  {1: [], 2: [(1, 1), (3, 1)], 3: [(4, 1)], 4: []}
Shortest path:  {1: 1, 2: 0, 3: 1, 4: 2}


2

In [153]:
times = [[1,2,1]]
n = 2
k = 1
networkDelayTime(times, n, k)

Graph:  {1: [(2, 1)], 2: []}
Shortest path:  {1: 0, 2: 1}


1

In [154]:
times = [[1,2,1]]
n = 2
k = 2
networkDelayTime(times, n, k)

Graph:  {1: [(2, 1)], 2: []}
Shortest path:  {1: inf, 2: 0}


-1

In [117]:
from typing import List
# time 똑같이 동시에 퍼진다고 가정하면 아래 코드로 풀기
def networkDelayTime(times: List[List[int]], n: int, k: int):
    # edge, weight 정보 담은 graph dict
    G = dict()
    # 1번 노드부터니까
    for i in range(1, n+1):
        G[i] = []
    
    # 정보 입력 
    for i in range(len(times)):
        from_ = times[i][0]
        to = times[i][1]
        weight = times[i][2]
        
        G[from_].append((to, weight))
    print(G)
    
    # bfs로 돌기
    q = []
    time = 0 # 시작 타임
    q.append((k, time)) # 시작 노드 넣기
    visited = [k] # 다 방문했는지 체크

    while q:
        cur, cur_time = q.pop(0) 
        print(cur, cur_time)
     
        if G[cur] == []: # 더이상 갈 곳이 없다면 다음 길로
            continue
        # 주변 노드와 해당 weight 꺼내기
        for adj, weight in G[cur]:
            # 아직 방문하지 않았다면 queue에 넣기
            if adj not in visited:
                # 같은 for loop은 동시에 뻗어감으로 새로운 time에 저장해서 넣기
                time = cur_time+weight
                visited.append(adj) # visited에 넣기
                q.append((adj, time))
                
    # 만약 다 visit하지 않았다면 불가능한 것이므로
    print(visited)
    if len(visited) != n:
        time = -1

    return time
                
    
    
    

In [118]:
times = [[1,2,1],[2,3,2],[1,3,2]]
n = 3
k = 1
networkDelayTime(times, n, k)

{1: [(2, 1), (3, 2)], 2: [(3, 2)], 3: []}
1 0
2 1
3 2
[1, 2, 3]


2

In [119]:
times = [[2,1,1],[2,3,1],[3,4,1]]
n = 4
k = 2
networkDelayTime(times, n, k)

{1: [], 2: [(1, 1), (3, 1)], 3: [(4, 1)], 4: []}
2 0
1 1
3 1
4 2
[2, 1, 3, 4]


2

In [120]:
times = [[1,2,1]]
n = 2
k = 1
networkDelayTime(times, n, k)

{1: [(2, 1)], 2: []}
1 0
2 1
[1, 2]


1

In [121]:
times = [[1,2,1]]
n = 2
k = 2
networkDelayTime(times, n, k)

{1: [(2, 1)], 2: []}
2 0
[2]


-1

# 210. Course Schedule II


In [ ]:
"""
[directed graph 순서 나열하기 (topological order)]

There are a total of numCourses courses you have to take, 
    labeled from 0 to numCourses - 1. 
You are given an array prerequisites 
    where prerequisites[i] = [ai, bi] indicates that 
    you must take course bi first if you want to take course ai.

For example, the pair [0, 1], indicates that to take course 0 
you have to first take course 1. (directed graph)
Return the ordering of courses you should take to finish all courses. 
If there are many valid answers, return any of them. 
If it is impossible to finish all courses, return an empty array.


Example 1:
Input: numCourses = 2, prerequisites = [[1,0]]
Output: [0,1]
Explanation: There are a total of 2 courses to take. To take course 1 you should have finished course 0. So the correct course order is [0,1].

Example 2:
Input: numCourses = 4, prerequisites = [[1,0],[2,0],[3,1],[3,2]]
Output: [0,2,1,3]
Explanation: There are a total of 4 courses to take. To take course 3 you should have finished both courses 1 and 2. Both courses 1 and 2 should be taken after you finished course 0.
So one correct course order is [0,1,2,3]. Another correct ordering is [0,2,1,3].

Example 3:
Input: numCourses = 1, prerequisites = []
Output: [0]
"""

In [13]:
from typing import List
def findOrder(numCourses: int, prerequisites: List[List[int]]) -> List[int]:
    G = dict()
    # number of courses만큼 graph adj list 만들기
    for i in range(numCourses):
        G[i] = []
        
    # 각 edge들 정보 추가하기 (directed)
    for i in range(len(prerequisites)):
        want = prerequisites[i][0]
        pre = prerequisites[i][1]
        
        G[pre].append(want)
        
    # topological sort 
    indegree = [0] * len(G) # 모든 노드의 진입차수를 0으로 초기화
    
    # 각 노드의 진입 차수 계산함
    """
    indegree 리스트는 각 노드의 초기 진입 차수를 저장하는 데 사용
    prerequisites = [[1,0],[2,0],[3,1],[3,2]]
    numCourses = 4

    이를 그래프로 나타내면 다음과 같습니다:
    0 -> 1
    0 -> 2
    1 -> 3
    2 -> 3

    따라서 진입 차수는 다음과 같습니다.
    0: 0
    1: 1
    2: 1
    3: 2
    """
    # 얼마나 들어오는 edges가 많은지
    for key in G:
        for neighbor in G[key]:
            indegree[neighbor] += 1
            
    Q = []
    for i in range(len(G)):
        if indegree[i] == 0:  # 진입차수가 0인 node를 큐에 넣는다.
            Q.append(i)
    
    order = [] # 위상 순서를 저장할 리스트

    while Q: # 큐가 비어있지 않으면 반복
        cur = Q.pop(0)
        order.append(cur) # 현재 node 리스트에 추가
        
        for adj in G[cur]: # 인접 노드 돌기
            indegree[adj] -= 1 # 인접 노드의 진입 차수 감소
            if indegree[adj] == 0: # 진입차수가 0이면 Q 추가
                Q.append(adj)
        

    # 모든 정점을 방문하지 않은 경우 (cycle 존재함)
    # 사이클을 형성하는 노드들 중 
    # 어느 노드도 진입 차수가 0이 되지 않기 때문에 큐에 들어가지 않게 됩니다.
    if len(order) != len(G):
        return []
    else:
        return order
    

In [14]:
numCourses = 2
prerequisites = [[1,0]]
findOrder(numCourses, prerequisites)

[0, 1]

In [15]:
numCourses = 4
prerequisites = [[1,0],[2,0],[3,1],[3,2]]
findOrder(numCourses, prerequisites)

[0, 1, 2, 3]

In [16]:
numCourses = 1
prerequisites = []
findOrder(numCourses, prerequisites)

[0]

# 1584. Min Cost to Connect All Points

<img src = "/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/스크린샷 2023-09-13 오후 2.31.36.png">

In [5]:
"""
[전체 node를 다 잇는 min cost 구하기]
You are given an array points representing integer coordinates of some points on a 2D-plane, 
    where points[i] = [xi, yi].

The cost of connecting two points [xi, yi] and [xj, yj] 
    is the manhattan distance between them: |xi - xj| + |yi - yj|, 
    where |val| denotes the absolute value of val.

Return the minimum cost to make all points connected. 
All points are connected if there is exactly one simple path between any two points.
"""
from typing import List, Tuple, Dict

def make_set(point, parents, keys):
    parents[point] = point
    keys[point] = 0

def find_set(point, parents):
    if parents[point] != point:
        parents[point] = find_set(parents[point], parents)
    # print(f"point : {point}, parents : {parents}")
    return parents[point]

def union(p1, p2, parents, keys):
    root1 = find_set(p1, parents)
    root2 = find_set(p2, parents)
    if root1 == root2:
        return
    if keys[root1] < keys[root2]:
        parents[root1] = root2
    elif keys[root1] > keys[root2]:
        parents[root2] = root1
    else:
        parents[root2] = root1
        keys[root1] += 1

def kruskal(graph:Dict[Tuple, Tuple]):
    ans = []
    parents, keys = dict(), dict()
    for v in graph.keys():
        make_set(v, parents, keys)
    
    # print(f"Before algorithm - parents : {parents}")
    # print(f"Before algorithm - keys : {keys}")
    
    edges = []
    for v in graph.keys():
        for edge in graph[v]:
            adj_point, adj_weight = edge
            edges.append((v, adj_point, adj_weight))
    
    print(f"Edges : {edges}")
    sorted_edges = sorted(edges, key=lambda x: x[-1])
    print(f"Sorted edges : {sorted_edges}")
    
    for edge in sorted_edges:
        u, v, weight = edge
        print(f"u: {u}, v: {v}, u.parent: {parents[u]}, v.parent: {parents[v]}, weight: {weight}")
        if find_set(u, parents) != find_set(v, parents):
            print(f"\tDifferent parents found.")
            ans.append(edge)
            union(u, v, parents, keys)
            print(f"\t{u} and {v} are connected.")
        else:
            print(f"\tConnected already, pass.")

    return ans, parents, keys

def minCostConnectPoints(points: List[List[int]]) -> int:
    def get_weight(p1, p2):
        p1_x, p1_y = p1
        p2_x, p2_y = p2
        return abs(p1_x - p2_x) + abs(p1_y - p2_y)

    graph = {}
    for point in points:
        graph[tuple(point)] = []
    
    for from_ in points:
        for to in points:
            if from_ == to:
                continue
            weight = get_weight(from_, to)
            graph[tuple(from_)].append((tuple(to), weight))
            graph[tuple(to)].append((tuple(from_), weight))
    print(f"Graph: {graph}")
    
    kruskal_edges, parents, keys = kruskal(graph)
    print(f"Result : {kruskal_edges}")
    # print(f"After algorithm - parents : {parents}")
    # print(f"After algorithm - keys : {keys}")
    
    total_cost = 0
    for edge in kruskal_edges:
        _, _, weight = edge
        total_cost += weight
    return total_cost


In [6]:
points = [[0,0],[2,2],[3,10],[5,2],[7,0]]
minCostConnectPoints(points)

Graph: {(0, 0): [((2, 2), 4), ((3, 10), 13), ((5, 2), 7), ((7, 0), 7), ((2, 2), 4), ((3, 10), 13), ((5, 2), 7), ((7, 0), 7)], (2, 2): [((0, 0), 4), ((0, 0), 4), ((3, 10), 9), ((5, 2), 3), ((7, 0), 7), ((3, 10), 9), ((5, 2), 3), ((7, 0), 7)], (3, 10): [((0, 0), 13), ((2, 2), 9), ((0, 0), 13), ((2, 2), 9), ((5, 2), 10), ((7, 0), 14), ((5, 2), 10), ((7, 0), 14)], (5, 2): [((0, 0), 7), ((2, 2), 3), ((3, 10), 10), ((0, 0), 7), ((2, 2), 3), ((3, 10), 10), ((7, 0), 4), ((7, 0), 4)], (7, 0): [((0, 0), 7), ((2, 2), 7), ((3, 10), 14), ((5, 2), 4), ((0, 0), 7), ((2, 2), 7), ((3, 10), 14), ((5, 2), 4)]}
Edges : [((0, 0), (2, 2), 4), ((0, 0), (3, 10), 13), ((0, 0), (5, 2), 7), ((0, 0), (7, 0), 7), ((0, 0), (2, 2), 4), ((0, 0), (3, 10), 13), ((0, 0), (5, 2), 7), ((0, 0), (7, 0), 7), ((2, 2), (0, 0), 4), ((2, 2), (0, 0), 4), ((2, 2), (3, 10), 9), ((2, 2), (5, 2), 3), ((2, 2), (7, 0), 7), ((2, 2), (3, 10), 9), ((2, 2), (5, 2), 3), ((2, 2), (7, 0), 7), ((3, 10), (0, 0), 13), ((3, 10), (2, 2), 9), ((3, 

20

# 1514. Path with Maximum Probability

<img src="/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/z_maxprob.png">

In [20]:
from typing import List

def maxProbability(n: int, edges: List[List[int]], succProb: List[float], start_node: int, end_node: int) -> float:

    # n = len(edges) # 노드 개수 (연결 안 된 node도 있어서 주어진 node 수 사용해야함)

    # 빈 그래프 생성 (G adj list dictionary 만들기)
    G = dict()
    # 노드 0부터 시작하므로 
    for i in range(0, n):
        G[i] = []
    
    
    # 주어진 간선 정보를 바탕으로 양방향 그래프 구성 (Undirected graph 정보 담기)   
    for i in range(len(edges)):
        from_ = edges[i][0] 
        to = edges[i][1] 
        # 해당 prob(weight) 정보와 함께 저장
        G[from_].append((to, succProb[i]))
        G[to].append((from_, succProb[i]))
    
    # shorteset path 찾는 함수 
    def shortestpath(G, s):
        # shortest distance 업뎃 정보 저장할 곳
        distances = dict()

        # distances 초기화 (max 곱하기 찾아야해서 -inf로)
        for key in G.keys():
            if key not in distances:
                distances[key] = -float('inf')
                
        # 곱하기여서 1로 초기화
        distances[s] = 1
        stack = [s]
        
        while stack:
            cur = stack.pop()
            # 주변 노드돌면서 해당 weight곱했을 때 더 크면 update하기
            for node, weight in G[cur]:
                if distances[node] < distances[cur] * weight:
                    distances[node] = distances[cur] * weight
                    stack.append(node)
        
        return distances
    
    dist = shortestpath(G, start_node)
    # end node가 업뎃 안 된거라면 연결 안 된 노드이므로 0 return
    if dist[end_node] == - float('inf'):
        return 0
    else:
        # 업뎃 된 최종적으로 큰 확률을 내보내기
        return dist[end_node]            

In [21]:
n = 3
edges = [[0,1],[1,2],[0,2]]
succProb = [0.5,0.5,0.3]
start = 0
end = 2

maxProbability(n, edges, succProb, start, end)

0.3

In [22]:
n = 3
edges = [[0,1],[1,2],[0,2]]
succProb = [0.5,0.5,0.2]
start = 0
end = 2
maxProbability(n, edges, succProb, start, end)

0.25

In [23]:
n = 3
edges = [[0,1]]
succProb = [0.5]
start = 0
end = 2
maxProbability(n, edges, succProb, start, end)

0

# 113. Clone Graph

In [ ]:
"""
graph 자체를 복사하는 방식 (노드들을 복사해서 adj list 내보내기)
"""

In [45]:
from collections import deque

class Node:
    def __init__(self, val=0, neighbors=None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []
        
# list를 graph 형식으로 (Node object) 만드는 함수 
def list_to_graph(adj_list):
    if not adj_list:
        return None

    # 값으로 Node 객체를 저장하는 사전을 만듭니다.
    node_dict = {}
    for val in range(1, len(adj_list) + 1):
        node_dict[val] = Node(val)

    for idx, neighbors in enumerate(adj_list, start=1):
        for neighbor_val in neighbors:
            node_dict[idx].neighbors.append(node_dict[neighbor_val])

    # 첫 번째 노드를 반환합니다.
    return node_dict[1]

# Node object를 담은 adj list를 print 하는 함수
def printGraph(node: 'Node'):
    # 방문한 노드를 기록하기 위한 집합 초기화
    visited = set()
    
    # 시작 노드를 큐에 넣기
    queue = deque([node])
    
    output = []
    # 큐에 노드가 있을 동안 반복
    while queue:
        # 큐의 가장 앞쪽에 있는 노드를 가져옴
        current = queue.popleft()
        
        # 현재 노드가 아직 방문되지 않았다면
        if current not in visited:
            # 현재 노드를 방문한 것으로 표시
            visited.add(current)
            
            # 현재 노드의 이웃들의 값을 저장할 리스트 초기화
            neighbors_vals = []

            # 현재 노드의 모든 이웃에 대하여
            for neighbor in current.neighbors:
                # 이웃의 값을 neighbors_vals 리스트에 추가
                neighbors_vals.append(neighbor.val)

            # 결과 리스트에 현재 노드의 이웃들의 값을 추가
            output.append(neighbors_vals)
            
            # # 현재 노드의 값과 이웃들의 값을 인접 리스트 형태로 출력 (1: 2, 4와 같은 형식)
            # print(f"{current.val}: {', '.join(map(str, neighbors_vals))}")

            # 현재 노드의 모든 이웃 노드에 대해서
            for neighbor in current.neighbors:
                # 만약 해당 이웃 노드가 아직 방문되지 않았다면
                if neighbor not in visited:
                    # 큐에 추가
                    queue.append(neighbor)
                    
    # 전체 결과를 인접 리스트 형태로 출력
    print(output)

"""
BFS 탐색을 할 때, 현재 노드의 모든 이웃 노드들을 살펴보고, 
    그 이웃 노드들이 이미 복제되었는지 mapping 사전을 통해 확인합니다. 
    
만약 복제되지 않았다면 새롭게 복제하고 큐에 추가합니다. 
    그러나 해당 이웃 노드가 이미 복제된 경우에는 큐에 다시 추가하지 않습니다.

이 방식 덕분에 순환 구조에서도 각 노드를 단 한 번만 방문하게 됩니다. 
    그래서 큐는 결국 비게 되고, BFS 탐색이 종료됩니다.
"""

def cloneGraph(node: 'Node') -> 'Node':
    if not node:
        return node

    # 원래 노드와 복제된 노드를 매핑할 사전을 만듭니다.
    mapping = {}

    # 시작 노드의 복제본을 만듭니다.
    mapping[node] = Node(node.val)

    # BFS 탐색을 위한 큐를 만듭니다.
    queue = deque([node])

    while queue:
        current = queue.popleft()
        
        for neighbor in current.neighbors:
            # 이웃 노드가 아직 복제되지 않았다면 복제하고 큐에 추가합니다.
            # 이를통해 cycle이어도 노드 한번만 체크하게 되는 것
            if neighbor not in mapping:
                mapping[neighbor] = Node(neighbor.val)
                queue.append(neighbor)

            # 현재 노드의 복제본의 이웃 리스트에 이웃 노드의 복제본을 추가합니다.
            mapping[current].neighbors.append(mapping[neighbor])

    # 복제된 그래프의 시작 노드로 그래프를 출력합니다.
    printGraph(mapping[node])
    
    # 시작 노드의 복제본을 반환합니다.
    return mapping[node]


In [46]:
adjList = [[2,4],[1,3],[2,4],[1,3]]
graph = list_to_graph(adjList)
cloned_graph = cloneGraph(graph)

[[2, 4], [1, 3], [1, 3], [2, 4]]


# 684. Redundant Connection

In [ ]:
"""
undirected graph (no cycle) 
중복된 edge return 하기 (여러개일 경우 맨 마지막 거 return)
해당 edge가 없어져도 모든 nodes이 다 연결되는 것

"""

<img src="/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/z_redundant.png">

In [36]:
# Cycle이 있는지 검사하고 cycle 경로를 반환하는 함수
"""
1) hasCycle 함수는 5개의 인자를 받습니다: 
    node (현재 노드), parent (현재 노드의 부모 노드), graph (그래프의 연결 정보), visited (노드의 방문 여부), path (현재까지의 경로).

2) 함수 시작 부분에서 현재 노드(node)는 방문된 것으로 표시되며, 경로(path)에 추가됩니다.

3) 그다음 현재 노드와 연결된 이웃 노드들을 순회하기 시작합니다.

4) 만약 이웃 노드가 현재 노드의 부모 노드(parent)와 다르다면, 
    그 이웃 노드에 대해 깊이 우선 탐색을 계속 진행합니다. 
    여기서 neighbor, node의 순서로 인자를 전달하는데, node는 다음 단계에서의 parent가 됩니다. 
    즉, 현재 노드가 다음 단계의 부모 노드로 업데이트되는 것입니다.

5) 만약 이웃 노드가 이미 방문된 상태거나, 
    그 이웃 노드에서 시작하는 깊이 우선 탐색에서 사이클이 발견되면 True를 반환하여 사이클이 존재함을 알립니다.

6) 만약 사이클을 발견하지 못했다면, 경로에서 현재 노드를 제거하고 False를 반환합니다.
"""
def hasCycle(node, parent, graph, visited, path):
    # 현재 노드를 방문 상태로 표시
    visited[node] = True
    # 현재 노드를 경로에 추가
    path.append(node)
    
    # 현재 노드와 연결된 이웃 노드들을 순회
    for neighbor in graph[node]:
        # 이웃 노드가 부모 노드(바로 직전에 방문한 노드)가 아닌 경우만 진행
        if neighbor != parent:
            # 이미 방문한 노드를 다시 방문한다면 cycle 존재
            if visited[neighbor] or hasCycle(neighbor, node, graph, visited, path):
                return True
    # 위의 조건에 해당하지 않는다면 경로에서 현재 노드를 제거
    path.pop()
    return False

# 주어진 간선들로부터 그래프에 cycle이 있는지 검사하고 cycle 경로를 반환
def findCycle(edges):
    
    n = len(edges) # 노드 개수 
    
    # 빈 그래프 생성 (G adj list dictionary 만들기)
    graph = {}
    # 노드 1부터 시작해서 
    for i in range(1, n+1):
        graph[i] = []

    # 주어진 간선 정보를 바탕으로 양방향 그래프 구성 (Undirected graph 정보 담기)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
        
    # 노드들의 방문 상태를 저장할 리스트 (node가 1부터 시작이니까)
    visited = [False] * (n + 1)
    # cycle 경로를 저장할 리스트
    path = []

    # 모든 노드에 대해 cycle이 있는지 검사 (node가 1부터 시작이니까)
    for i in range(1, n+1):
        # 먼저 parent는 -1로 설정
        if not visited[i] and hasCycle(i, -1, graph, visited, path):
            return path

    return []

def findRedundantConnection(edges):

    paths = findCycle(edges)
    if len(paths) == 0:
        return []

    i = 0
    redundant = []
    while (i != len(edges)):
        true_cnt = 0
        for idx in range(len(paths)):
            if paths[idx] in edges[i]:
                true_cnt += 1
        
        # edge (두 노드) 모두 cycle에 포함이면 cycle 만드는 edge
        if true_cnt == 2: 
            redundant.append(edges[i])
            true_cnt = 0 # reset
            # print(redundant) # [[1, 2], [2, 3], [3, 4], [1, 4]]
        i += 1
        
    return redundant[-1]
            
            

if __name__ == "__main__":
    edges = [[1,2],[2,3],[3,4],[1,4],[1,5]]
    print(findRedundantConnection(edges))  # 출력: [1, 4]
    # print(findCycle(edges)) # [1, 2, 3, 4]
    
    


[1, 4]


In [37]:
edges = [[1,2],[1,3],[2,3]]
findRedundantConnection(edges) # 출력: [2, 3]


[2, 3]

In [11]:
from typing import List
# Node definition
class GNode:
    def __init__(self, id, color="W", p=None):
        self.id = id
        self.color = color
        self.parent = p
    
    def __str__(self): # object를 넣었을 때 어떤 걸 프린트할지
        return self.id 

def dfs_cycle(G,s):
    # 그래프 모든 노드 초기화
    for key in G:
        key.color = 'W'
        key.parent = None
        
     # 시작 노드의 색상과 거리 설정
    s.color = "G"
    stack = [s] # DFS에 source node 넣고 시작
    iscycle = False
    while stack:
        cur = stack.pop()
        for adj in G[cur]:
            if adj.color == 'G' and adj.parent != cur:
                iscycle = True
                return iscycle
            
            if adj.color == 'W': # 방문 전이라면
                adj.color = 'G' # 방문 표시
                adj.parent = cur # 부모 현재노드로 바꾸기
                stack.append(adj)         
                 
        cur.color = 'B'
        return iscycle

# GNode인 경우의 dict 프린트하기
def print_G(G):
    for key in G:
        print(key.id, ":", [node.id for node in G[key]]) 

# node가 숫자일 경우 해당 숫자와 GNode형태 매칭할 dict 만들기 
def nodeDict(edges):
    nodes = set()
    for edge in edges:
        nodes.add(edge[0])
        nodes.add(edge[1])
    # print(nodes)
    
    node_dict = {}
    for node in nodes:
        node_dict[node] = GNode(id=node)
    
    return node_dict

def findRedundantConnection(edges:List[List[int]]):
    # 숫자 노드와 GNode 매칭할 dict 사용
    node_dict = nodeDict(edges)
    
    G = dict() # undireced graph edges 정보 담을 adj list
    for i in range(len(edges)):
        from_ = node_dict[edges[i][0]]
        to = node_dict[edges[i][1]]
        if from_ not in G:
            G[from_] = [to]
        else:
            G[from_].append(to)
            
        if to not in G:
            G[to] = [from_]
        else:
            G[to].append(from_)
            
    # Gnode 담은 dict print하기
    print_G(G)

    # cycle이 발견될때 dfs 멈추기 (source node 바꾸면서 )
    for key in G:
        iscycle = dfs_cycle(G,key)
        if(iscycle):
            break
    cycle = []
    for key in G:
        if key.color == 'G':
            cycle.append(key.id)
    
    print(cycle)
            
        
            
        
        


In [12]:
edges = [[1,2],[1,3],[2,3]]
findRedundantConnection(edges)

1 : [2, 3]
2 : [1, 3]
3 : [1, 2]
[1, 2]


# 1. All Paths from source to target (DAG)

a directed edge from node i to node graph[i][j] 

In [20]:
# 0부터 모든 노드까지 가능한 Paths 반환하기
def allPathsSourceTarget(graph):
    
    # graph 정보 있는 Dictionary 만들기
    G = dict()
    for i in range(len(graph)):
            G[i] = graph[i]
    # print(G)
    
    # 각 노드와 해당 노드까지의 경로 저장
    stack = [(0, [0])]
    allpaths = []
    # Acyclic path 찾아야 하므로 DFS 이용
    while stack:
        cur, path = stack.pop()
        
        # 더이상 갈 노드가 없다면 (목적지라면)
        if len(G[cur]) == 0:
            # print('stack: ', stack)
            allpaths.append(path)
            continue
    
        for adj in G[cur]:
            if adj not in path:
                stack.append((adj, path+[adj]))
    return allpaths
    
    

In [18]:
graph = [[1,2],[3],[3],[]]
allPathsSourceTarget(graph)

{0: [1, 2], 1: [3], 2: [3], 3: []}
stack:  [(1, [0, 1])]
stack:  []


[[0, 2, 3], [0, 1, 3]]

In [21]:
graph = [[4,3,1],[3,2,4],[3],[4],[]]
allPathsSourceTarget(graph)

{0: [4, 3, 1], 1: [3, 2, 4], 2: [3], 3: [4], 4: []}
stack:  [(4, [0, 4]), (3, [0, 3]), (3, [0, 1, 3]), (2, [0, 1, 2])]
stack:  [(4, [0, 4]), (3, [0, 3]), (3, [0, 1, 3])]
stack:  [(4, [0, 4]), (3, [0, 3])]
stack:  [(4, [0, 4])]
stack:  []


[[0, 1, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 4]]

# 1334. Find the City With the Smallest Number of Neighbors at a Threshold

<img src = "/Users/soungmunkim/Desktop/Coding/QualExamCoding/soungmunkim/Python/Graph/z_img/z_문제스샷.png">

In [7]:
import sys
import numpy as np

# 연결되지 않은 path의 weight
INF = sys.maxsize

 #--------------------- Floyd warhshall 알고리즘 구현 (모든 source node에 대해 shortest path -----------------------
# Floyd-Warshall Algorithm
# input - adj(인접행렬), n(vertex의 수)
# output - dist : 최소 i->j의 path를 기록한 matrix (shortest path distance 기록)
# output - pred : i->j의 path에서 j 직전에 거치는 node를 기록한 matrix. (shortest path node기록)
def Floyd_warshall(adj,n):
    # 거리 기록할 배열 초기화
    dist = adj

    # predecessor 기록할 배열 초기화
    pred = [['NIL']*n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j and adj[i][j] != INF:
                pred[i][j] = i

    # intermediate node 'k'를 하나씩 추가하며, 해당 경우의 최단 거리 계산
    for k in range(n):
        for i in range(n):   # i: 출발노드
            for j in range(n):   # j: 도착노드

                # k를 추가함으로써 이전보다 더 빠른 길이 생기는 경우의 처리
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]
    return dist, pred

# shortest path list에 담아 출력하기
def print_shortest_path(pred, i, j):
    path = []
    def recursion(pred, i, j):
        if i == j:
            # print(i, end=' ')
            path.append(i)
            return
        recursion(pred, i, pred[i][j])
        # print(j, end=' ')
        path.append(j)   
    
    recursion(pred, i, j)
    return path
    

def findTheCity(n, edges, distanceThreshold):
    # Row, Col = len(edges), len(edges)
    adj = np.zeros((n, n))
    
    #--------------------- bidirectical adj.matrix 만들기-----------------------
    for i in range(n): # 한 방향
        for j in range(n):
            # 먼저 같은 노드가 아니면 INf.로 initialize 하기
            if i != j:
                adj[i][j] = float('inf')
            
            from_node = edges[i][0]
            to_node = edges[i][1]
            dist = edges[i][2]
            adj[from_node][to_node] = dist
            
    for i in range(n): # 다른 한 방향
        for j in range(n):
            if i != j:
    
                from_node = edges[i][1]
                to_node = edges[i][0]
                dist = edges[i][2]
                adj[from_node][to_node] = dist
    #----------------------------------------------------------------------------             
            
    # print("adj. matrix: \n", adj) # 완성된 bidrectical adj.matrix
    
    distance, pred = Floyd_warshall(adj, n)  # 여기서 Path는 프린트 할 필요 없음 (pred 사용 안 함)
    # print("distance matrix: \n", distance)  
    print("pred matrix: \n", pred)
    
    shortestpaths = dict()
    allpaths = []
    for i in range(len(distance)):
        for j in range(len(distance[0])):
            # i -> j까지의 최단 거리이므로
        
            if i != j and distance[i][j] <= distanceThreshold:
                # shortest path에 각 Source마다 shortest distance가 4이하인 도착점 넣기
                print((i,j), end='\t')
                if i not in shortestpaths:
                    shortestpaths[i] = [j]
                else:
                    shortestpaths[i].append(j)
                print(print_shortest_path(pred, i, j))
                # allpaths.append(print_shortest_path(pred, i, j))
                
                    
    # print("all paths: ", allpaths)
    
    min_num = 100
    max_city = -1
    # print("shortestpath: \n", shortestpaths)
    
    # 가장 적은 number of cities중 가장 큰 Source node return 하기
    for city, adj in shortestpaths.items():
        if len(adj) <= min_num:
            min_num = len(adj)
            if city > max_city:
                max_city = city
    
    return max_city     
    

In [8]:
n = 4
edges = [[0,1,3],[1,2,1],[1,3,4],[2,3,1]]
distanceThreshold = 4
findTheCity(n, edges, distanceThreshold)

pred matrix: 
 [['NIL', 0, 1, 2], [1, 'NIL', 1, 2], [1, 2, 'NIL', 2], [1, 2, 3, 'NIL']]
(0, 1)	[0, 1]
(0, 2)	[0, 1, 2]
(1, 0)	[1, 0]
(1, 2)	[1, 2]
(1, 3)	[1, 2, 3]
(2, 0)	[2, 1, 0]
(2, 1)	[2, 1]
(2, 3)	[2, 3]
(3, 1)	[3, 2, 1]
(3, 2)	[3, 2]


3